In [ ]:
# This plots our RR Lyrae stars in galactic coordinates, so as to see where they are relative
# to the SDSS Ca absorption footprint (Murga+ 2015 MNRAS 452, 511)

# created 2018 July 8 by E.S.

In [1]:
import numpy as np
import pandas as pd
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.io import fits

In [2]:
# read data on our RRLs
stars = pd.read_csv('rrl_gal_coords.csv')

In [3]:
# define read-in data as degrees
gal_lon_l_all = coord.Angle(stars['gal_lon_l']*u.degree)         
gal_lat_b_all = coord.Angle(stars['gal_lat_b']*u.degree) 
gal_lon_l_ab = coord.Angle(stars['gal_lon_l'].loc[stars['type']=='ab']*u.degree)         
gal_lat_b_ab = coord.Angle(stars['gal_lat_b'].loc[stars['type']=='ab']*u.degree) 
gal_lon_l_c = coord.Angle(stars['gal_lon_l'].loc[stars['type']=='c']*u.degree)         
gal_lat_b_c = coord.Angle(stars['gal_lat_b'].loc[stars['type']=='c']*u.degree) 

In [4]:
# make coordinates
stars_coords_all = coord.Galactic(l=gal_lon_l_all,b=gal_lat_b_all)
stars_coords_ab = coord.Galactic(l=gal_lon_l_ab,b=gal_lat_b_ab)
stars_coords_c = coord.Galactic(l=gal_lon_l_c,b=gal_lat_b_c)

In [5]:
# read in Ca data
image, header = fits.getdata('maps_EW(CaNa)_20150318.fits',0,header=True)

In [6]:
# convert Ca absorption data into coordinates
cak_lon_l = coord.Angle(image['longitude']*u.degree)
cak_lat_b = coord.Angle(image['latitude']*u.degree)
cak_ew = image['Ca_K_EW ']
cak_coords = coord.Galactic(l=cak_lon_l,b=cak_lat_b)

In [7]:
# replace zeros with nans
cak_ew[cak_ew == 0] = 'nan'

In [10]:
plt.clf()
fig = plt.figure()
ax = fig.add_subplot(111, projection="mollweide")
#plt.scatter(image['longitude']*np.pi/180., image['latitude']*np.pi/180., c=cak_ew, cmap=cm.Reds)
plt.scatter(cak_coords.l.wrap_at(180*u.deg).rad, cak_coords.b.wrap_at(180*u.deg).rad, 
            c=cak_ew, vmin=0, vmax=0.5, edgecolors='none')
plt.plot(stars_coords_ab.l.wrap_at(180*u.deg).rad, stars_coords_ab.b.wrap_at(180*u.deg).rad, 'o', color='k', markersize=10, markeredgewidth=4)
plt.plot(stars_coords_c.l.wrap_at(180*u.deg).rad, stars_coords_c.b.wrap_at(180*u.deg).rad, 'x', color='red', markersize=10, markeredgewidth=4)

for i, txt in enumerate(stars['name']):
    ax.annotate(stars['name'][i], (stars_coords_all.l.wrap_at(180*u.deg).rad[i],stars_coords_all.b.wrap_at(180*u.deg).rad[i]))

#plt.gca().invert_xaxis()
ax.grid(True)
plt.suptitle('CaK absorption coverage (Murga+ 2015, esp. Fig. 2); black dots = our RRabs; xs = our RRcs')
plt.xlabel('galactic l')
plt.ylabel('galactic b')
plt.colorbar()
plt.show()